In [1]:
import numpy as np
import math
import tensorflow as tf

In [2]:
def trapezoidIntegral(vals, ds):
    pass1 = tf.reduce_sum(vals, axis=0)
    pass2 = tf.reduce_sum(vals[1:-1], axis=0)
    return ds/2.0*(pass1+pass2)
def TFconv(image, filt, b):
    return tf.nn.convolution(input=image, filters=filt, padding="SAME") + b

In [108]:
def loopTFuSoln(raw_u0, ds, inFilt, pFilt, b):
    t1 = inFilt.shape[0]
    input_shape = raw_u0.shape
    bzero = tf.zeros([inFilt.shape[4]])
    u = tf.Variable(tf.zeros((t1,*input_shape)))
    x = tf.Variable(tf.zeros((t1,*input_shape)))
    u[0].assign(TFconv(raw_u0, inFilt[0], bzero))
    
    for h in range(t1-1):
        x[h].assign(tf.nn.leaky_relu(u[h], alpha=0.1))
        zInt = tf.Variable(tf.zeros((h+1,*input_shape)))
        for s in range(h+1):
            zInt[s].assign(TFconv(x[s], pFilt[s,h], b[h] / ((h+1)*ds)))
        z = trapezoidIntegral(zInt, ds)
        u[h+1].assign(z + TFconv(raw_u0, inFilt[h+1], bzero))
    x[-1].assign(tf.nn.leaky_relu(u[-1], alpha=0.1))
    
    return x,u 

In [102]:
def TFuSoln(raw_u0, ds, inFilt, pFilt, b):
    bzero = tf.zeros([inFilt.shape[4]])
    u0 = TFconv(raw_u0, inFilt[0], bzero)
    #for loop begins, h=0
    x0 = tf.nn.leaky_relu(u0, alpha=0.1)
    zInt0 = TFconv(x0, pFilt[0,0], b[0] / ((0+1)*ds))
    #print(zInt0)
    z0 = trapezoidIntegral(zInt0, ds)
    u1 = z0 + TFconv(raw_u0, inFilt[1], bzero)
    #h=1
    x1 = tf.nn.leaky_relu(u1, alpha=0.1)
    zInt1_0 = TFconv(x0, pFilt[0,1], b[1] / ((1+1)*ds)) #s=0
    zInt1_1 = TFconv(x1, pFilt[1,1], b[1] / ((1+1)*ds)) #s=1
    #zInt1 = zInt1_0 + zInt1_1
    zInt1 = tf.stack([zInt1_0, zInt1_1])
    z1 = trapezoidIntegral(zInt1, ds)
    u2 = z1 + TFconv(raw_u0, inFilt[2], bzero)
    #h=2
    x2 = tf.nn.leaky_relu(u2, alpha=0.1)#
    zInt2_0 = TFconv(x0, pFilt[0,2], b[2] / ((2+1)*ds)) #s=0
    zInt2_1 = TFconv(x1, pFilt[1,2], b[2] / ((2+1)*ds)) #s=1
    zInt2_2 = TFconv(x2, pFilt[2,2], b[2] / ((2+1)*ds)) #s=2
    #zInt2 = zInt2_0 + zInt2_1 + zInt2_2
    zInt2 = tf.stack([zInt2_0, zInt2_1, zInt2_2])
    z2 = trapezoidIntegral(zInt2, ds)#
    u3 = z2 + TFconv(raw_u0, inFilt[3], bzero)
    #h=3
    x3 = tf.nn.leaky_relu(u3, alpha=0.1)
    zInt3_0 = TFconv(x0, pFilt[0,3], b[3] / ((3+1)*ds)) #s=0
    zInt3_1 = TFconv(x1, pFilt[1,3], b[3] / ((3+1)*ds)) #s=1
    zInt3_2 = TFconv(x2, pFilt[2,3], b[3] / ((3+1)*ds)) #s=2
    zInt3_3 = TFconv(x3, pFilt[3,3], b[3] / ((3+1)*ds)) #s=3
    #zInt3 = zInt3_0 + zInt3_1 + zInt3_2 + zInt3_3
    zInt3 = tf.stack([zInt3_0, zInt3_1, zInt3_2, zInt3_3])
    z3 = trapezoidIntegral(zInt3, ds)
    u4 = z3 + TFconv(raw_u0, inFilt[4], bzero)
    #end loop
    x4 = tf.nn.leaky_relu(u4, alpha=0.1)
    
    #print("z with h=0", z0)
    #print("z with h=1", z1)
    #print("z with h=2", z2)
    #print("z with h=3", z3)
    
    x = tf.stack([x0,x1,x2,x3,x4])
    u = tf.stack([u0,u1,u2,u3,u4])
    
    return x,u

In [85]:
t0 = 4
t1 = t0+1
dt = 1.0/t0
input_shape = (1,5,5)
filter_shape = (1,1,3,3)



actWt = [.1, 1] #leakyRelU: first number is slope left of origin, second number is slope right of origin

np.set_printoptions(suppress=True)
rng = np.random.default_rng(42)
uIn = rng.standard_normal(input_shape, dtype=np.float32)
iFilt = rng.standard_normal((t1,*filter_shape), dtype=np.float32)
paramFilt = rng.standard_normal((t1,t1,*filter_shape), dtype=np.float32)
biasVec = rng.standard_normal((t1,1), dtype=np.float32)

#xOut, uOut = uSoln(uIn, dt, iFilt, paramFilt, biasVec)
#print(xOut)

In [103]:
TFuIn = tf.expand_dims(tf.transpose(tf.convert_to_tensor(uIn, dtype=tf.float32), perm=[1,2,0]), 0)
TFiFilt = tf.transpose(tf.convert_to_tensor(iFilt, dtype=tf.float32), perm=[0,3,4,1,2])
TFpFilt = tf.transpose(tf.convert_to_tensor(paramFilt, dtype=tf.float32), perm=[0,1,4,5,2,3])
TFbiasVec = tf.convert_to_tensor(biasVec, dtype=tf.float32)

TFxOut, TFuOut = TFuSoln(TFuIn, dt, TFiFilt, TFpFilt, TFbiasVec)
print(tf.transpose(TFuOut, perm=[4,0,1,2,3]))

tf.Tensor(
[[[[[ 0.9516136   2.435678   -0.3856678  -1.6703204   0.00031823]
    [ 2.8851762   2.1393359  -3.9642057  -1.1248233   0.65912706]
    [ 0.5324     -2.6901844   0.19213542  1.9467492   0.17811385]
    [-3.6203117   1.7217662   4.8276005   2.2568848  -2.2175238 ]
    [ 0.72034025  0.01188321  0.37456763 -2.4593573  -0.16356802]]]


  [[[-2.9307792  -1.2281874  -3.417582   -1.8477119   2.026555  ]
    [-2.9081187  -1.6186329   2.366631    5.005061   -0.46067193]
    [-0.9149097  -3.2843504   5.979022    2.6312397  -0.51302314]
    [ 2.1489542   4.835612   -8.582668   -4.3518667  -1.8402613 ]
    [-0.5486518  -1.8495516   0.8251563   2.3737555   5.4625373 ]]]


  [[[ 1.192028    6.2110214   3.1337578  -0.8437642  -1.5232902 ]
    [-7.1727448  -5.9361267   0.54477304  2.7715528   1.641727  ]
    [-1.3377907  -4.409808   -2.7376275   4.135856    2.1630063 ]
    [ 1.0291543   2.04671     2.0405087   1.282228   -0.43683746]
    [-2.746736   -0.38681155 -2.9954038  -1.8713902  -1.3

In [107]:
loopTFxOut, loopTFuOut = loopTFuSoln(TFuIn, dt, TFiFilt, TFpFilt, TFbiasVec)
print(tf.transpose(loopTFuOut, perm=[4,0,1,2,3]))

(1, 5, 5, 1)
tf.Tensor(
[[[[[ 0.9516136   2.435678   -0.3856678  -1.6703204   0.00031823]
    [ 2.8851762   2.1393359  -3.9642057  -1.1248233   0.65912706]
    [ 0.5324     -2.6901844   0.19213542  1.9467492   0.17811385]
    [-3.6203117   1.7217662   4.8276005   2.2568848  -2.2175238 ]
    [ 0.72034025  0.01188321  0.37456763 -2.4593573  -0.16356802]]]


  [[[-2.9307792  -1.2281874  -3.417582   -1.8477119   2.026555  ]
    [-2.9081187  -1.6186329   2.366631    5.005061   -0.46067193]
    [-0.9149097  -3.2843504   5.979022    2.6312397  -0.51302314]
    [ 2.1489542   4.835612   -8.582668   -4.3518667  -1.8402613 ]
    [-0.5486518  -1.8495516   0.8251563   2.3737555   5.4625373 ]]]


  [[[ 1.192028    6.2110214   3.1337578  -0.8437642  -1.5232902 ]
    [-7.1727448  -5.9361267   0.54477304  2.7715528   1.641727  ]
    [-1.3377907  -4.409808   -2.7376275   4.135856    2.1630063 ]
    [ 1.0291543   2.04671     2.0405087   1.282228   -0.43683746]
    [-2.746736   -0.38681155 -2.9954038  -1.